In [ ]:
#| default_exp lake_utils

### Utilities

Functions for Common tasks

#### Unzipping from S3 to S3

In [ ]:
#| exporti

def unzip(lake_interface, source_file, destination_folder, exclude_pattern=None, include_pattern=None):
    logs = [
        '-' * 30,
        f'Copying from {source_file} to {destination_folder}'
    ]
    
        
    zip_obj = lake_interface.get_object(source_file)
    buffer = BytesIO(zip_obj["Body"].read())

    z = zipfile.ZipFile(buffer)
    file_names = z.namelist()
    filtered_files = file_names.copy()

    if include_pattern:
        filtered_files = [f for f in filtered_files if include_pattern in f]

    if exclude_pattern:
        filtered_files = [f for f in filtered_files if exclude_pattern not in f]

    for filename in filtered_files:
        file_info = z.getinfo(filename)

        lake_interface.s3.upload_fileobj(
            Fileobj=z.open(filename),
            Bucket=lake_interface.bucket,
            Key=f'{destination_folder}/{filename}'
        )

        logs.append(f'Copied {filename}')

    return logs


#### Fetching Glue Data Catalog

In [ ]:
#| exporti

def parse_column_info(cols):
    return [
        {
            'column_name': c['Name'], 
            'column_type': c['Type']
        } for c in cols if '__index_level' not in c['Name']
    ]
    
def parse_table_info(raw_table):
    return {
        'table_name': raw_table['Name'],
        'database_name': raw_table['DatabaseName'],
        'location': raw_table['StorageDescriptor']['Location'],
        'columns': parse_column_info(raw_table['StorageDescriptor']['Columns'])
    }

In [ ]:
#| export

def get_data_catalog(lake_interface, database_name):
    """
    Fetches catalog for glue database

    Parameters
    ----------
    lake_interface: LakeInterface, required
        instance of class providing access to datalake
    database_name : str, required
        Name of AWS Glue database

    Returns
    -------
    table_columns : list(str)
        List of all columns in every table in form [table_name].[column_name]

    """
    #harvest aws crawler metadata

    session = lake_interface.session
    glue = session.client('glue')

    next_token = ""
    #glue = boto3.client('glue',region_name='us-east-1')
    tables = []

    while True:
        resp = glue.get_tables(DatabaseName=database_name, NextToken=next_token)

        for tbl in resp['TableList']:
            tables.append(parse_table_info(tbl))
        next_token = resp.get('NextToken')
        
        if next_token is None:
            break

    return tables
